<a href="https://colab.research.google.com/github/vnewsome/project_chd/blob/main/ML_project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### DS3001: Machine Learning Project 2


In [1]:
! git clone https://github.com/vnewsome/project_chd

Cloning into 'project_chd'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 16 (delta 4), reused 10 (delta 3), pack-reused 2
Receiving objects: 100% (16/16), 644.81 KiB | 10.57 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [2]:
# load import statements
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# load the training data
data = pd.read_csv("/content/project_chd/fhs_train.csv", low_memory=False)

In [4]:
# examining data
data.head()

,Unnamed: 0,sex,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1267,1,58,1.0,0,0.0,0.0,0,0,0,220.0,143.0,104.0,29.85,75,87.0,1
1,1209,0,40,1.0,1,15.0,0.0,0,0,0,199.0,122.0,82.0,22.16,85,77.0,0
2,2050,0,52,1.0,0,0.0,0.0,0,0,0,275.0,112.0,71.0,25.68,80,NaN,0
3,1183,1,38,2.0,1,43.0,0.0,0,1,0,170.0,130.0,94.0,23.90,110,75.0,0
4,3225,0,43,1.0,0,0.0,0.0,0,0,0,202.0,124.0,92.0,21.26,75,74.0,0


In [14]:
# looking at each variable, cleaning if necessary

column_dtypes = data.dtypes   # examining the data types
print(column_dtypes)

# variables that contain nans: education, cigsPerDay, BPMeds, totChol, glucose
glucose = data["glucose"]
glucose.unique()

Unnamed: 0           int64
sex                  int64
age                  int64
education          float64
currentSmoker        int64
cigsPerDay         float64
BPMeds             float64
prevalentStroke      int64
prevalentHyp         int64
diabetes             int64
totChol            float64
sysBP              float64
diaBP              float64
BMI                float64
heartRate            int64
glucose            float64
TenYearCHD           int64
dtype: object


array([ 87.,  77.,  nan,  75.,  74., 104.,  69.,  90.,  91.,  81.,  89.,
        86.,  84.,  73.,  95.,  65., 103.,  76.,  85.,  92.,  72.,  62.,
        93.,  68.,  67., 122., 102.,  79.,  66.,  83.,  78.,  64.,  63.,
        58.,  71., 115.,  70.,  96.,  60.,  82.,  94.,  88., 167., 117.,
       215.,  48., 108., 268., 100.,  80.,  99., 135.,  97.,  55., 274.,
        40., 118.,  47.,  56., 112., 248., 107., 105.,  61., 260., 170.,
       148., 120., 225.,  98.,  59., 332., 110., 206., 129.,  57., 114.,
       101., 113., 111., 127., 121.,  45., 147.,  50., 244., 106., 394.,
       173., 116., 137., 145., 123.,  44., 172.,  54., 142., 348., 119.,
       320.,  53., 136., 210.,  43., 144., 292., 125., 294., 130., 131.,
       255., 160.,  52., 207., 297., 216., 166., 109., 126., 235., 140.,
       270., 250., 193., 256., 150., 202., 254., 370., 191., 186.])

In [15]:
# determining how many NaNs are in the data
nan_columns = data.isnull().sum()
print(nan_columns)
total_columns = data.count()
print(total_columns)

# glucose is the only column with a high percentage of nans (~9%)

Unnamed: 0           0
sex                  0
age                  0
education           85
currentSmoker        0
cigsPerDay          24
BPMeds              37
prevalentStroke      0
prevalentHyp         0
diabetes             0
totChol             39
sysBP                0
diaBP                0
BMI                 15
heartRate            0
glucose            285
TenYearCHD           0
dtype: int64
Unnamed: 0         3180
sex                3180
age                3180
education          3095
currentSmoker      3180
cigsPerDay         3156
BPMeds             3143
prevalentStroke    3180
prevalentHyp       3180
diabetes           3180
totChol            3141
sysBP              3180
diaBP              3180
BMI                3165
heartRate          3180
glucose            2895
TenYearCHD         3180
dtype: int64


In [16]:
# examining relationships between columns before building a model

# sex variable: 1 = male
# TenYearCHD: assuming 1 indicates an at risk individual (note: this was not noted in the data dictionary)
data.loc[:,['totChol','TenYearCHD'] ].groupby('TenYearCHD').describe()     # totChol and 10-year risk of CHD have a higher mean when compared to no risk

totChol                                                          
             count        mean        std    min    25%    50%    75%    max
TenYearCHD                                                                  
0           2660.0  234.699624  43.708817  113.0  204.0  232.0  260.0  696.0
1            481.0  247.114345  49.429131  107.0  214.0  243.0  273.0  600.0

In [20]:
data.loc[:,['BMI','TenYearCHD'] ].groupby('TenYearCHD').describe()

BMI                                                        
             count       mean       std    min    25%    50%    75%    max
TenYearCHD                                                                
0           2686.0  25.761102  4.029037  15.54  23.08  25.35  27.96  51.28
1            479.0  26.600355  4.598560  16.71  23.68  26.20  28.92  56.80

In [18]:
data.loc[:,['cigsPerDay','TenYearCHD'] ].groupby('TenYearCHD').describe()

cigsPerDay                                                 
                count       mean        std  min  25%  50%   75%   max
TenYearCHD                                                            
0              2671.0   8.726320  11.789079  0.0  0.0  0.0  20.0  70.0
1               485.0  10.315464  13.114147  0.0  0.0  0.0  20.0  60.0

In [21]:
data.loc[:,['age','TenYearCHD'] ].groupby('TenYearCHD').describe()     # possible relationship bw age and ten year CHD risk

age                                                   
             count       mean       std   min   25%   50%   75%   max
TenYearCHD                                                           
0           2693.0  48.813219  8.405781  32.0  42.0  48.0  55.0  70.0
1            487.0  54.172485  7.954541  35.0  48.0  55.0  61.0  70.0